# **Recurrent** **Neural** **Network** **Model**

#### Environment Configurartion

In [ ]:
!nvidia-smi #GPU Enabled 

Fri Apr 30 16:30:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# we want the latest tensorflow, and we do not want tensorboard because to clashes when we want to visualise
try:
  !pip uninstall tb-nightly tensorboardX 
  !pip install tf-nightly
except:
  pass

     |████████████████████████████████| 456.8MB 36kB/s 
     |████████████████████████████████| 4.2MB 40.7MB/s 
     |████████████████████████████████| 1.3MB 33.7MB/s 
     |████████████████████████████████| 5.9MB 35.1MB/s 
     |████████████████████████████████| 4.0MB 35.7MB/s 
     |████████████████████████████████| 471kB 43.1MB/s 
     |████████████████████████████████| 3.9MB 33.1MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: h5py 2.10.0
    Uninstalli

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import tensorflow as tf

import os
import datetime
import tensorflow_datasets as tfds


%load_ext tensorboard

In [ ]:
import pkg_resources

for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
  print(entry_point.dist)

tensorboard 2.4.1
tensorboard-plugin-wit 1.8.0
tb-nightly 2.6.0a20210430


In [ ]:
!rm -r /usr/local/lib/python3.6/dist-packages/tensorboardcolab-0.0.22.dist-info

rm: cannot remove '/usr/local/lib/python3.6/dist-packages/tensorboardcolab-0.0.22.dist-info': No such file or directory


In [ ]:
print(tf.__version__)

2.6.0-dev20210430


In [ ]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

##### Load dataset from Amazon

In [ ]:
dataset, info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', with_info=True)
train_dataset = dataset['train']

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompleteZIEDTE/amazon_us_reviews-train.tfrecord


Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


In [ ]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [ ]:
print(train_dataset)

<PrefetchDataset shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


In [ ]:
len(list(train_dataset))

104975

In [ ]:
BUFFER_SIZE = 30000 # preload the dataset into the model, speed the model
BATCH_SIZE = 128 # We will use Mini batch gradient descent to train our model, 128 for each step

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'13067811'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Mobile_Electronics'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B002UNSOLW'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'244088816'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'CrazyOnDigital Premium Pink iPod Nano 5th Generation Leather Case. CrazyOnDigital Retail Package'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'Great little cover for the price.  Just a warning it you have to take the cover off in order to charge/plug in your ipod.  Other than that is does what it needs to do.'>, 'review_date': <tf.Tensor: shape=(), dtype=string, numpy=b'2011-11-14'>, 'review_headline': <tf.Tensor: shape=(), dtype=string, numpy=b'Great for the price'>, 'review_i

In [ ]:
for reviews in train_dataset.take(10):
  reviews_text = reviews['data']
  print(reviews_text.get('review_body').numpy()) # we can store text in tensors and numpy
  print(reviews_text.get('star_rating'))
  print(tf.where(reviews_text.get('star_rating')>3,1,0).numpy())

b'Great little cover for the price.  Just a warning it you have to take the cover off in order to charge/plug in your ipod.  Other than that is does what it needs to do.'
tf.Tensor(3, shape=(), dtype=int32)
0
b'Fits like the original...'
tf.Tensor(5, shape=(), dtype=int32)
1
b"I bought this impulsively because of the sheer cuteness of it. After using it for a week, I have decided to return it. The design is bulky, and you don't even get to enjoy the cute turtle because it is on the back. It fits too loosely for my comfort, and it seems that it would stretch after time. I felt that it was pretty thick, and could be pretty durable. The power button was hard to push as there is a thick piece right over it. I cannot here it click, and I would sometimes have to press it three times to put the Ipod to sleep. The texture of the case picks up dust and dirt like a magnet. After the week I used it, it looked as if I have had it for a year. My Ipod doesn't move around much, it only goes in my car

In [ ]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary = set() # remove duplicates
for _, reviews in train_dataset.enumerate():
  review_text = reviews['data']
  reviews_tokens = tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary.update(reviews_tokens)

vocab_size = len(vocabulary)
vocab_size

73738

In [ ]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary) # We create interger values for our words

In [ ]:
print(vocabulary) # you can preprocess this words, 

{'multifunction', 'mortar', 'Proper', 'Printer', '445', 'lithium', 'luse', 'mantain', 'liquids', 'klunky', 'SPLITTER', 'maintanance', 'workspace', 'Karmen', 'leaving', '1900x1080', 'goodbyebuddy', 'Surveillance', 'U1', 'Crosse', 'matted', 'wiki', 'BX8s', 'grabber', 'studs', 'XBI', 'involves', 'unglued', 'disclaim', 'Produced', 'Changed', 'H8', 'VideoLan', 'Solidly', 'horseshoe', 'Absolutely', 'int', 'mankind', '5G', 'Myonly', 'inslall', 'discreto', 'Reccomended', 'eninme', 'unlress', 'somtlething', 'bookcover', 'Tractor', 'phobia', 'violated', 'sparkels', 'Spotlifter', 'Celtic', 'ENVIO', 'bucks', 'exports', 'pIayer', '8700C', 'DJs', 'attractions', 'Victory', 'BUZZER', 'Seeinstripes', 'track', 'othes', 'KIDDING', 'INSTRUCTIONS', 'VE', 'Buena', 'pilots', 'boomboxers', 'MUST', 'thers', 'Headphone', '9pm', 'génial', 'thenif', 'proctected', 'Goos', 'VDO', 'polaris', 'ARLiberator', 'polarization', 'inatvertantly', 'comin', 'planted', 'Ventilator', 'oscillating', 'colntrol', 'streamer', '6944

In [ ]:
for reviews in train_dataset.take(5):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())
  encoded_example = encoder.encode(review_text.get('review_body').numpy())
  print(encoded_example)

b'Great little cover for the price.  Just a warning it you have to take the cover off in order to charge/plug in your ipod.  Other than that is does what it needs to do.'
[64640, 3618, 17370, 70166, 17127, 33538, 44646, 63104, 13950, 10716, 69856, 53574, 57994, 49081, 17127, 17370, 52461, 29166, 25467, 57994, 29170, 38432, 29166, 68816, 7927, 18271, 4063, 67772, 59220, 21558, 50690, 10716, 29035, 57994, 51242]
b'Fits like the original...'
[4865, 69844, 17127, 13381]
b"I bought this impulsively because of the sheer cuteness of it. After using it for a week, I have decided to return it. The design is bulky, and you don't even get to enjoy the cute turtle because it is on the back. It fits too loosely for my comfort, and it seems that it would stretch after time. I felt that it was pretty thick, and could be pretty durable. The power button was hard to push as there is a thick piece right over it. I cannot here it click, and I would sometimes have to press it three times to put the Ipod t

In [ ]:
for index in encoded_example:
  print('{}----->{}'.format(index, encoder.decode([index])))

55418----->Thank
38788----->God
61775----->we
29929----->were
17114----->home
14670----->when
39007----->this
19118----->filter
48218----->cracked
52555----->It
9818----->flooded
17127----->the
62894----->fridge
11422----->hardwood
16125----->kitchen
63515----->floor
68834----->Entire
65097----->sheetrock
45106----->ceiling
19557----->and
54866----->family
6194----->room
31283----->carpet
24912----->below
44646----->Just
25842----->called
17127----->the
58472----->insurance
12435----->company
2603----->What
63104----->a
9636----->bad
2921----->day
55881----->Never
39290----->buy
39007----->this


In [ ]:
# Please note that the reason we using tensorflow, pandas is memory bouneded, it cannot work with lot of data

In [ ]:
def encode(text_tensor, label_tensor):
  encoded_text = encoder.encode(text_tensor.numpy())
  label = tf.where(label_tensor>3, 1, 0)
  return encoded_text, label

In [ ]:
def encode_map_fn(tensor):

  text = tensor['data'].get('review_body')
  label = tensor['data'].get('star_rating')

  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label],
                                       Tout=(tf.int64, tf.int32))
  
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

In [ ]:
ar_encoded_data = train_dataset.map(encode_map_fn)

In [ ]:
for f0, f1 in ar_encoded_data.take(2):
  print(f0)
  print(f1)

tf.Tensor(
[64640  3618 17370 70166 17127 33538 44646 63104 13950 10716 69856 53574
 57994 49081 17127 17370 52461 29166 25467 57994 29170 38432 29166 68816
  7927 18271  4063 67772 59220 21558 50690 10716 29035 57994 51242], shape=(35,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor([ 4865 69844 17127 13381], shape=(4,), dtype=int64)
tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
# SPLIT THE DATASET INTO TRAIN AND TEST

TAKE_SIZE = 10000

train_data = ar_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE) # we are padding the text, so that all the text can look like similar length

test_data = ar_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [ ]:
vocab_size += 1

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(837,), dtype=int64, numpy=
 array([64640,  3618, 17370, 70166, 17127, 33538, 44646, 63104, 13950,
        10716, 69856, 53574, 57994, 49081, 17127, 17370, 52461, 29166,
        25467, 57994, 29170, 38432, 29166, 68816,  7927, 18271,  4063,
        67772, 59220, 21558, 50690, 10716, 29035, 57994, 51242,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

In [ ]:
for f0, f1 in test_data.take(10):
  print(tf.unique_with_counts(f1)[2].numpy())


[46 82]
[40 88]
[82 46]
[88 40]
[89 39]
[47 81]
[80 48]
[83 45]
[92 36]
[83 45]


In [ ]:
# Lets create the Model

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 128)) # embedding layers (words are represented in dense vector because they are mnore) / and when is sparse vector words is onlu 0 and 1
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))# we want the model to undertand the whole sentences, so we inlcude the bidirectional layer in out lstm, train model foward and backward to the data
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))) # Bidirectional create two direction back and foward to undertsnad the whole sentence
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))
  #model.add(tf.kears.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(1))

  # RNN is good for sequential data such as time series. langauge translation

In [ ]:
!rm -r /tmp/logs/

rm: cannot remove '/tmp/logs/': No such file or directory


In [ ]:
logdir = os.path.join("/tmp/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="/tmp/sentiment_analysis.hdf5", verbose=1, save_best_only=True)

model.compile(optimizer='adam', 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, epochs=5, validation_data=test_data, callbacks=[tensorboard_callback, checkpointer])

# Increase the complexity of the model to get better accuracy, and regularization/dropout to reduce overfitting

259/742 [=========>....................] - ETA: 1:17:31 - loss: 0.4881 - accuracy: 0.7510

KeyboardInterrupt: ignored

In [ ]:
model.save("/tmp/final_sentiment_analysis.hdf5")

In [ ]:
!ls -alrt /tmp/*.hdf5

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print(eval_loss)
print(eval_acc)

In [ ]:
for f0, f1 in test_data.take(1):
  print(f1)
  print(model.predict(f0))

In [ ]:
model.layers

In [ ]:
model.summary()

In [ ]:
!nvidia-smi

In [ ]:
# Use matplotlib to visualise the performance, check: https://www.youtube.com/watch?v=1aBPXWLftFs&list=PL3N9eeOlCrP6zMkHMxFJV4yXIsET5aWlc / continue